In [2]:
import pandas as pd
import sqlite3

In [3]:
pages = pd.read_excel(
    "C:\\Users\\Clinton Ukachukwu\\Desktop\\DA Projects\\Wikipedia\\wikipedia_dataset_flat.xlsx",
    sheet_name="Sheet1",
    header=0
)

In [4]:
pages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 549000 entries, 0 to 548999
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   Unnamed: 0  549000 non-null  int64         
 1   date        549000 non-null  datetime64[ns]
 2   page        549000 non-null  object        
 3   visits      548913 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 16.8+ MB


In [3]:
pages.head()

,Unnamed: 0,date,page,visits
0,0,2016-01-01,.xxx_en.wikipedia.org_all-access_all-agents,7089.0
1,1,2016-01-01,.xxx_en.wikipedia.org_mobile-web_all-agents,6182.0
2,2,2016-01-01,1._Juli_de.wikipedia.org_desktop_all-agents,20.0
3,3,2016-01-01,2014_FIFA_World_Cup_en.wikipedia.org_all-acces...,5363.0
4,4,2016-01-01,2014_FIFA_World_Cup_en.wikipedia.org_desktop_a...,3186.0


In [4]:
pages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 549000 entries, 0 to 548999
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   Unnamed: 0  549000 non-null  int64         
 1   date        549000 non-null  datetime64[ns]
 2   page        549000 non-null  object        
 3   visits      548913 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 16.8+ MB


In [5]:
pages=pages.dropna()

In [6]:
pages=pages.drop_duplicates()

In [7]:
pages.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 548913 entries, 0 to 548999
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   Unnamed: 0  548913 non-null  int64         
 1   date        548913 non-null  datetime64[ns]
 2   page        548913 non-null  object        
 3   visits      548913 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 20.9+ MB


Creating a SQLite Database

In [8]:
db_conn = sqlite3.connect("C:\\Users\\Clinton Ukachukwu\\Desktop\\DA Projects\\Wikipedia\\wikipedia_dataset_flat.db")

In [9]:
c = db_conn.cursor()

In [10]:
# PAGES
c.execute(
    """
    CREATE TABLE pages (
        date TEXT,
        page TEXT,
        visits INTEGER
        
        );
     """
)

In [11]:
pages = pages.drop(columns=['Unnamed: 0'])
pages.to_sql('pages', db_conn, if_exists='append', index=False)

548913

Proportion of languages represented in Wikipedia dataset

In [14]:
### Extract language code from page names
pages['language'] = pages['page'].str.extract(r'_(\w+).wikipedia.org')

In [15]:
### Count languages represented in the page column
num_languages = pages['language'].nunique()
print(f"Number of languages: {num_languages}")

Number of languages: 533


In [26]:
pages['corrected_language'] = pages['language'].str.extract(r'\b([a-z]{2})\b')


In [27]:
### Calculate proportion of pages for each language
language_counts = pages['corrected_language'].value_counts()
language_proportions = language_counts / len(pages)
print(language_proportions * 100)  # This will display the proportions as percentages

en    22.654592
ja     7.867731
es     2.467058
ru     2.333703
de     2.066994
zh     1.866963
fr     1.333545
Name: corrected_language, dtype: float64


In [28]:
language_proportions_pages = language_proportions.reset_index()
language_proportions_pages.page = ['Language', 'Proportion']
language_proportions_pages.to_csv('wikipedia_language_proportions.csv', index=False)


C:\Users\Clinton Ukachukwu\AppData\Local\Temp\ipykernel_13016\148216540.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  language_proportions_pages.page = ['Language', 'Proportion']


Device type more frequently used for visiting Wikipedia

In [20]:
### Extract device type from page column
pages['device_type'] = pages['page'].str.extract(r'_(desktop|mobile)')

In [24]:
### Verify the extraction
print(pages['device_type'].value_counts())

mobile     107604
desktop     99100
Name: device_type, dtype: int64


In [21]:
### Aggregate the data
device_counts = pages['device_type'].value_counts()

In [25]:
print(device_counts)

mobile     107604
desktop     99100
Name: device_type, dtype: int64


In [22]:
### Export to CSV
device_counts.to_csv('wikipedia_device_usage.csv')

In [23]:
### Verify the export
with open('wikipedia_device_usage.csv', 'r') as file:
    print(file.readline())
    print(file.readline())

,device_type

mobile,107604

